In [64]:
import xgboost as xgb
from random import randint
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np


In [65]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 14 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: bool(2), float64(11), int64(1)
memory usage: 3.9+ MB


In [66]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################


df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('data/event_por_hora.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos = df_unidos.join(df16).fillna(0)

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 96 columns):
cant_conversions                                38829 non-null float64
cant_checkouts                                  38829 non-null float64
cant_viewed_product                             38829 non-null float64
cant_searched_product                           38829 non-null float64
cant_ad_campaign_hit                            38829 non-null float64
ad_campaign_hit                                 38829 non-null bool
cant_lead                                       38829 non-null float64
lead                                            38829 non-null bool
cant_brand_listing                              38829 non-null float64
cant_visited_site                               38829 non-null float64
cant_generic_listing                            38829 non-null float64
total_sesiones                                  38829 non-null int64
promedio_eventos_por_sesion               

In [67]:
fea = list(df_unidos)
print len(fea)


lista = []
def crear_list_features():
    for i in range(21):
        num = randint(0,95)
        feas = fea[num]
        while feas in lista:
            num = randint(0,95)
            feas = fea[num]
        lista.append(feas)
crear_list_features()


lista



96


['hour_13',
 'sku',
 'staticpage_how-to-sell',
 'hour_19',
 'staticpage_black_friday',
 'compro_samsung',
 'hour_21',
 'city_S\xc3\xa3o Paulo',
 'conte_10',
 'staticpage_CustomerService',
 'channel_Direct',
 'device_type_Unknown',
 'hour_7',
 'cant_visited_site',
 'total_sesiones',
 'event_checkout',
 'hour_17',
 'channel_Paid',
 'promedio_eventos_por_sesion',
 'compro_apple',
 'hour_20']

In [68]:
['event_viewed product','hour_19','cant_checkouts','prom_timestamp',
 'compro_samsung','ult_view','event_search engine hit','device_type_Smartphone','staticpage_TermsAndConditionsReturnEcommerce','hour_9','check_sku',
 'total_sesiones','channel_Paid','hour_0','staticpage_FaqEcommerce','hour_22','hour_1','ad_campaign_hit','channel_Organic','hour_23','staticpage_how-to-sell']

['event_viewed product',
 'hour_19',
 'cant_checkouts',
 'prom_timestamp',
 'compro_samsung',
 'ult_view',
 'event_search engine hit',
 'device_type_Smartphone',
 'staticpage_TermsAndConditionsReturnEcommerce',
 'hour_9',
 'check_sku',
 'total_sesiones',
 'channel_Paid',
 'hour_0',
 'staticpage_FaqEcommerce',
 'hour_22',
 'hour_1',
 'ad_campaign_hit',
 'channel_Organic',
 'hour_23',
 'staticpage_how-to-sell']




['event_viewed product',
 'hour_19',
 'cant_checkouts',
 'prom_timestamp',
 'compro_samsung',
 'ult_view',
 'event_search engine hit',
 'device_type_Smartphone',
 'staticpage_TermsAndConditionsReturnEcommerce',
 'hour_9',
 'check_sku',
 'total_sesiones',
 'channel_Paid',
 'hour_0',
 'staticpage_FaqEcommerce',
 'hour_22',
 'hour_1',
 'ad_campaign_hit',
 'channel_Organic',
 'hour_23',
 'staticpage_how-to-sell']

In [69]:
df_unidos = df_unidos[lista]
df_unidos

,hour_13,sku,staticpage_how-to-sell,hour_19,staticpage_black_friday,compro_samsung,hour_21,city_São Paulo,conte_10,staticpage_CustomerService,...,device_type_Unknown,hour_7,cant_visited_site,total_sesiones,event_checkout,hour_17,channel_Paid,promedio_eventos_por_sesion,compro_apple,hour_20
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0,9288.0,0,0,0,0.0,0,0.0,True,0,...,0,0,1.0,1,0,0,0,9.000000,0.0,0
ad93850f,0,5904.0,0,0,0,0.0,0,1.0,True,0,...,0,0,5.0,5,0,0,0,13.000000,0.0,0
0297fc1e,1,2832.0,0,1,0,0.0,1,0.0,True,0,...,0,0,95.0,89,0,1,0,6.370787,0.0,1
2d681dd8,0,10084.0,0,1,0,0.0,0,0.0,False,0,...,0,0,2.0,2,1,0,0,13.000000,0.0,0
cccea85e,1,10407.0,0,1,0,0.0,1,0.0,True,0,...,0,0,22.0,17,0,0,0,49.176471,0.0,1
4c8a8b93,0,6873.0,0,0,0,0.0,1,0.0,True,0,...,0,0,20.0,14,0,0,0,18.357143,0.0,0
1b9f7cf6,0,9987.0,0,1,0,0.0,0,0.0,True,0,...,0,0,3.0,3,0,0,0,8.333333,0.0,0
29ebb414,0,7153.0,0,0,0,0.0,0,0.0,True,0,...,0,0,4.0,4,0,1,0,8.750000,0.0,0
de8fe91b,0,10337.0,0,0,0,0.0,0,1.0,True,0,...,0,0,2.0,2,0,0,0,26.500000,0.0,0


In [70]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')


df_test = df_unidos.join(labels, how= "inner")
df_test.head()

,hour_13,sku,staticpage_how-to-sell,hour_19,staticpage_black_friday,compro_samsung,hour_21,city_São Paulo,conte_10,staticpage_CustomerService,...,hour_7,cant_visited_site,total_sesiones,event_checkout,hour_17,channel_Paid,promedio_eventos_por_sesion,compro_apple,hour_20,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,0,5904.0,0,0,0,0.0,0,1.0,True,0,...,0,5.0,5,0,0,0,13.000000,0.0,0,0
1b9f7cf6,0,9987.0,0,1,0,0.0,0,0.0,True,0,...,0,3.0,3,0,0,0,8.333333,0.0,0,0
de8fe91b,0,10337.0,0,0,0,0.0,0,1.0,True,0,...,0,2.0,2,0,0,0,26.500000,0.0,0,0
45baf068,0,3371.0,0,0,0,0.0,0,0.0,True,0,...,0,1.0,1,0,0,0,20.000000,0.0,0,0
99abca5a,1,8567.0,0,0,0,0.0,1,0.0,True,0,...,0,56.0,49,0,1,0,16.714286,0.0,1,0


In [71]:
X, y = df_test.iloc[:,:-1],df_test.iloc[:,-1]  #X van los features, en y el label, asumo que esta bien esto

In [72]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [74]:
#y_train

In [75]:
param_grid = { 
    'n_estimators': [x for x in range(200, 550, 50)],
    'learning_rate':[0.01],
    'max_depth' : [2,3,4],
    #'gamma':[0,0.5,1,1.5,2,2.5,3],
    
    'scale_pos_weight': [x for x in range(2, 6, 1)]
} 

In [76]:
from sklearn.model_selection import GridSearchCV 
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10) 
CV_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2 
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, score=0.669794236796, total=   3.9s
[CV] n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, score=0.640479127767, total=   3.9s
[CV] n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max_depth=2, score=0.660713057291, total=   4.0s
[CV] n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=2, learning_rate=0.01, max

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    9.2s


[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, score=0.639935232729, total=   4.9s
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, score=0.669033617298, total=   5.1s
[CV] n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2 
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, score=0.671478959119, total=   5.0s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, score=0.663909786975, total=   5.4s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=3, learning_rate=0.01, max_depth=2, score=0.692491704526, total=   5.3s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2 


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   15.0s


[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, score=0.669854669579, total=   5.3s
[CV] n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, score=0.641426255335, total=   5.4s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, score=0.679900731485, total=   4.9s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, score=0.668660583812, total=   4.7s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=4, learning_rate=0.01, max_depth=2, score=0.692715084549, total=   4.6s
[CV] n_estimators=200, scale_pos_weight=5, learning_rate=0.01, max_depth=2 
[CV]  n_estimators=200, scale_pos_weight=5, learning_ra

...

[CV]  n_estimators=450, scale_pos_weight=2, learning_rate=0.01, max_depth=4, score=0.703488713541, total=  13.1s
[CV] n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4, score=0.673320177047, total=  13.0s
[CV] n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4, score=0.644342658042, total=  13.0s
[CV] n_estimators=450, scale_pos_weight=4, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4, score=0.701109422587, total=  13.0s
[CV] n_estimators=450, scale_pos_weight=4, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=450, scale_pos_weight=3, learning_rate=0.01, max_depth=4, score=0.670018698836, total=  13.1s
[CV] n_estimators=450, scale_pos_weight=4, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=450, scale_pos_weight=3, learning_ra

[Parallel(n_jobs=-1)]: Done 413 tasks      | elapsed: 15.7min


[CV]  n_estimators=500, scale_pos_weight=4, learning_rate=0.01, max_depth=4, score=0.670096871228, total=  14.0s
[CV] n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=500, scale_pos_weight=4, learning_rate=0.01, max_depth=4, score=0.709127748204, total=  14.0s
[CV] n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4, score=0.674700753534, total=  14.0s
[CV] n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4 
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4, score=0.640723984729, total=  13.9s
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4, score=0.696698415081, total=  13.8s
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4, score=0.672000108399, total=  13.7s
[CV]  n_estimators=500, scale_pos_weight=5, learning_rate=0.01, max_depth=4, score=0.709264083

[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed: 16.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [200, 250, 300, 350, 400, 450, 500], 'scale_pos_weight': [2, 3, 4, 5], 'learning_rate': [0.01], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [77]:
CV_xgb.best_params_ #aca te dice el mejor hiperparametro


{'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 500,
 'scale_pos_weight': 2}

In [78]:
xg_reg = XGBClassifier(gamma = 5,max_depth=3,learning_rate=0.01,n_estimators=500,scale_pos_weight = 5,silent=True,n_jobs=-1)
#falta jugar con los hiperparametros
# Voting con multinomialNB, RF, y SVC (no lineal), probar las distintas combinaciones

In [79]:
xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=5, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [80]:
score=xg_reg.score(X_test,y_test)
print(score)

Y_pred = xg_reg.predict(X_test)
print(roc_auc_score(y_test, Y_pred))
#1  0.887458150914


"""0.940252382179
0.563060150205"""

0.954674220963
0.507287422491


'0.940252382179\n0.563060150205'

In [55]:
feature_importances = pd.DataFrame(xg_reg.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(40)

,importance
ult_view,0.483465
cant_generic_listing,0.153644
check_samsung,0.123500
cant_ad_campaign_hit,0.110916
hour_10,0.033655
hour_3,0.031021
hour_21,0.025461
event_search engine hit,0.013169
hour_18,0.012584
hour_6,0.005268


In [30]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')


kaggle_pred = xg_reg.predict_proba(df_events)
proba_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_comprar)
df_submit['label'] = series.values

df_submit.to_csv('submit/submit_xg_2_3.csv')

In [31]:
len(df_events)

19415

In [32]:
df_submit["label"].mean()

0.17564859186178503

In [33]:
CV_xgb

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=80, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [200, 250, 300, 350, 400, 450, 500], 'scale_pos_weight': [2, 3, 4, 5], 'learning_rate': [0.01], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [34]:
df_submit_2 = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events_2 = df_submit_2.join(df_unidos, how='inner')


kaggle_pred_2 = CV_xgb.predict_proba(df_events_2)
proba_comprar_2 = [x[1] for x in kaggle_pred_2]
series = pd.Series(proba_comprar)
df_submit_2['label'] = series.values

df_submit_2.to_csv('submit/submit_xg_2_2.csv')

In [35]:
df_submit

,label
person,
4886f805,0.031302
0297fc1e,0.345921
2d681dd8,0.033875
cccea85e,0.249663
4c8a8b93,0.094040
29ebb414,0.058524
3dc1950f,0.240569
8ea4c165,0.134579
d8cfe234,0.145017


In [36]:
df_submit_2["label"].mean()

0.17564859186178503